<table style="width:100%">
  <tr style="background-color:#FFFFFF">
    <td style="width:10%">
        <img src="https://play-lh.googleusercontent.com/K9FC23HoIYy_idt3LL26zrzMchOGORVzLNmi3L5u-MaghdjpdgHAWv7wQlscNmo9wYjC=s180-rw" alt="DiDi" width=120></td>
    <td style="width:10%">
        <img src="https://logodownload.org/wp-content/uploads/2019/05/99-app-logo.png" alt="99" width=120></td>
      <td>
          <h1 style="text-align:left; text-indent :2em;">DATA TRANSFORMATION</h1>
          <h2 style="text-align:left; text-indent :2.4em;">ISIS SANTOS COSTA • 2020</h2>
      </td>
  </tr>
</table>

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Setting the DataFrames

In [1]:
import pandas as pd

In [2]:
df_orders = pd.read_csv('orders.csv')
df_trips = pd.read_csv('trips.csv')

### Taking a first look

In [3]:
df_orders.head(1)

,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD
0,6433698,5234567812367422,2014-04-12 00:20:28,-73.958403,40.713247,CRD


In [4]:
df_trips.head(1)

,718300,2014-05-24 09:08:43,2014-05-24 09:19:06,2.1,-73.967172,40.769851,1,-73.995022,40.765441,CRD,9.5,2.5,12.5,5234567815585627,5234567812346342
0,14054197,2014-04-10 15:14:12,2014-04-10 15:49:05,12.4,-73.991044,40.749704,1,-73.909173,40.8235,CSH,39.0,0.0,39.5,5234567812372271,5234567812379863


### Naming the columns

In [5]:
df_orders_cols = ['order_id','passenger_id','pickup_datetime','pickup_longitude','pickup_latitude','payment_type']
df_orders = pd.read_csv('orders.csv', names = df_orders_cols)

In [6]:
df_orders.head(1)

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD


In [7]:
df_trips_cols = ['order_id','pickup_datetime','dropoff_datetime','trip_distance','pickup_longitude','pickup_latitude','rate_code','dropoff_longitude','dropoff_latitude','payment_type','fare_amount','tip_amount','total_amount','passenger_id1','passenger_id2']
df_trips = pd.read_csv('trips.csv', names = df_trips_cols)

In [8]:
df_trips.head(1)

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,total_amount,passenger_id1,passenger_id2
0,718300,2014-05-24 09:08:43,2014-05-24 09:19:06,2.1,-73.967172,40.769851,1,-73.995022,40.765441,CRD,9.5,2.5,12.5,5234567815585627,5234567812346342


### Basic inter-table consistency

<i>Note: There should be a single passenger_id for each trip, buuuut... there are two! </i>🤔<i> Hm...
<br> Well... let's check which one is matches the one for the same order_id in 'orders', and drop the other.</i>


In [9]:
df_orders.loc[df_orders['order_id'] == 6433697]

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD


In [10]:
df_trips.loc[df_trips['order_id'] == 6433697]

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,total_amount,passenger_id1,passenger_id2
4218614,6433697,2014-04-11 18:06:45,2014-04-11 18:18:19,1.6,-74.015684,40.714476,1,-74.000814,40.718173,CRD,9.5,2.2,13.2,5234567816269547,5234567812347954


👀 <i>We see that the 1<sup>st</sup> passenger ID is the one equal to that of the orders table, so let's rename it and drop the other.</i>

In [11]:
df_trips.rename(columns = {'passenger_id1': 'passenger_id'}, inplace = True)
df_trips.drop('passenger_id2', inplace = True, axis = 1)
df_trips.head(1)

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,total_amount,passenger_id
0,718300,2014-05-24 09:08:43,2014-05-24 09:19:06,2.1,-73.967172,40.769851,1,-73.995022,40.765441,CRD,9.5,2.5,12.5,5234567815585627


<i>Nice! Better now</i> 🙂  <i>Time to start checking closer.</i> 🔎

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Checking & improving the data structure

### Trips x Orders

In [12]:
df_trips.shape

(11456987, 14)

In [13]:
df_orders.shape

(15000000, 6)

<i>From the length of the tables, we can get a 1<sup>st</sup> interesting number, the worst-case for the trips lost:</i> 
📏

In [14]:
print('Potential trips lost, worst-case:',  '\033[1m' + '{:.0%}'.format(1-df_trips.shape[0]/df_orders.shape[0]))

Potential trips lost, worst-case: 24%


 ( ◕︵◕ ) <i> That seems quite a lot... To be checked later.</i>

### Payment type

In [15]:
100 * df_orders['payment_type'].value_counts() / len(df_orders)

CRD    57.618107
CSH    41.549827
UNK     0.495213
NOC     0.258573
DIS     0.078280
Name: payment_type, dtype: float64

In [16]:
100 * df_trips['payment_type'].value_counts() / len(df_trips)

CRD    57.602518
CSH    41.564523
UNK     0.496186
NOC     0.258419
DIS     0.078354
Name: payment_type, dtype: float64

### Rate type

In [17]:
100 * df_trips['rate_code'].value_counts() / len(df_trips)

1      97.725632
2       1.812117
5       0.266711
3       0.155983
4       0.031570
0       0.006363
6       0.001353
210     0.000166
8       0.000052
65      0.000026
208     0.000017
28      0.000009
Name: rate_code, dtype: float64

<i> Codes were expected to be 1-6 and there is no mapping for the meaning of other values. </i> Being their frequency ultra low, let's just drop them. 🧹

In [18]:
df_trips_trsf = df_trips.loc[df_trips['rate_code'] <= 6] # df_trips transformed (removed code > 6)
100 * df_trips_trsf['rate_code'].value_counts() / len(df_trips_trsf) # % frequency of rate types

1    97.725897
2     1.812122
5     0.266711
3     0.155984
4     0.031570
0     0.006363
6     0.001353
Name: rate_code, dtype: float64

In [19]:
aux = df_trips_trsf
df_trips_trsf = aux.loc[aux['rate_code'] != 0] # df_trips_trsf transformed (removed code = 0)
100 * df_trips_trsf['rate_code'].value_counts() / len(df_trips_trsf) # % frequency of rate types

1    97.732115
2     1.812237
5     0.266728
3     0.155994
4     0.031572
6     0.001353
Name: rate_code, dtype: float64

In [20]:
removed = df_trips.shape[0] - df_trips_trsf.shape[0]
removed_pct = 100 * removed / df_trips.shape[0]
txt = 'N° of trips purged given to extraneous type of rate: '
print(txt, '\033[1m' + str(removed), ' (', '{:.6f}'.format(removed_pct), '%)', sep='')

N° of trips purged given to extraneous type of rate: 760 (0.006634%)


## Exporting results for retrieval

In [21]:
df_orders.to_csv('df_orders.csv')

In [23]:
df_trips_trsf.to_csv('df_trips.csv')

<i>Note: This Notebook would entail the whole transforming process, but as some instabilities may interrupt the Kernel requiring a restart from zero, it has been split for a smoothier process.</i>

## Continues as [Transformation, part 2](99__DataTransformation2.ipynb)